In [1]:
from manim import *
import numpy as np
from src.grid import Grid
from src.vector_field import VectorField
from src.utils.curve_generator import ellipse_function
from src.animation.animate_field import build_vector_field_scene
from src.utils.numerical_methods import closest_point_on_curve, numerical_tangent_function, closest_point_on_surface


In [2]:
%load_ext manim

The manim module is not an IPython extension.


# Curves and surfaces with differential forms

## Introduction

This is the second part of our exploration of the Plateau problem and the approach taken to solve it in the paper by Wang and Chern.

In this part, we will focus on defining the problem using vector fields and differential forms. As the original paper authors state, doing this reformalization allows us to consider the problem as a convex optimization problem, which makes finding the correct solution easier and more stable. Formulating the problem in this matter does require us however to go through a couple of mathematical definitions and examples. We do this in order to obtain a comprehensive overview of the solution method.

We begin by discussion of vector fields on smooth manifolds, then move on to differential forms and then finally restate the Plateau problem using objects known as dirac-delta differential forms.

## Vector fields

The basic object of study is a vector field. Usually vector fields are first introduced as simply functions on some euclidean space (2 or 3 -dimensional) which map an "arrow" to each point in space. Formally, for example a 2-dimensional vector field is a function $f: \mathbb{R}^2 \to \mathbb{R}^2$, usually with some notion of smoothness included as well to make the function behave _nicely_.

The basic definition works well in an Euclidean setting and allows us to define some other operators, such as the divergence and curl operators (in suitable dimensions). Furthermore, it allows a simple mental picture of what is going on. One can image the speed and direction of wind on the globe as a vector field defined on the 2-sphere, as the prototypical example.

However, in more general context of differential geometry and smooth manifolds, this basic definition does not make sense. In the sense that if we don't have global coordinates for our space, but simply a local coordinate chart at each point $p$ on the smooth manifold $M$, then we cannot define this vector field globally for each point on the surface. 

To remedy this, we can modify the definition of a vector field to be more abstract using things called _derivations_.

### Vector fields as derivations

At each point $p \in M$, we consider all the smooth real-valued functions on $M$, denoted $C^\infty(M)$. A tangent vector at $p$ can then be thought of as a **directional derivative operator** acting on these functions. That is, a vector $X_p \in T_pM$ is defined as a linear map
$X_p : C^\infty(M) \to \mathbb{R}$ which satisfies the **Leibniz rule**:

$$
X_p(fg) = X_p(f)g(p) + f(p)X_p(g)
$$

This is exactly the rule you expect from a derivative operator—it’s how the derivative of a product behaves. Such a map is called a **derivation**. A vector field $X$ is then the map from a point $p \in M$ to this linear map $X_p$. Formally, $X: p \mapsto X_p$. And the application of vector field to a smooth function produces a map from a point to a real number, $X(f): p \mapsto \mathbb{R}$.

### Why is this useful?

This approach gives us a definition of vectors that:

- **Works on any smooth manifold**, whether or not we have coordinates.
- Is **intrinsic**, i.e., does not rely on choosing coordinates or embedding the manifold in some higher-dimensional space.
- Naturally leads to the dual concept of differential forms, which are linear functionals on vectors.


### Coordinate Basis Vectors and Derivations

In $\mathbb{R}^n$, we usually express a vector as a linear combination of the standard basis vectors: $v = a^1 e_1 + a^2 e_2 + \dots + a^n e_n$, where each $e_i$ points in the direction of the $x^i$-axis. On a smooth manifold $M$, there is no global coordinate system, but around any point $p \in M$, we can choose a local coordinate chart $(x^1, \dots, x^n)$. In this chart, we define the **coordinate vector fields** $\left( \frac{\partial}{\partial x^1}, \dots, \frac{\partial}{\partial x^n} \right)$ using differentiation: $\frac{\partial}{\partial x^i} (f)(p) = \frac{\partial f}{\partial x^i}(p)$. In other words, a coordinate vector field simply differentiates the input function $f \in C^\infty(M)$ at point $p$ in the "direction" of the specific coordinate, leaving other coordinates alone, exactly the partial derivative by definition.

We define the **tangent space** $T_pM$ to be the set of all derivations at $p$. The collection $\left( \frac{\partial}{\partial x^1} \right)_p, \dots, \left( \frac{\partial}{\partial x^n} \right)_p$ forms a **basis** of $T_pM$. Therefore, any vector $X_p \in T_pM$ can be uniquely expressed as some linear combination $X_p = a^1 \left( \frac{\partial}{\partial x^1} \right)_p + \dots + a^n \left( \frac{\partial}{\partial x^n} \right)_p$ and acts on functions as a directional derivative:
$X_p(f) = a^1 \frac{\partial f}{\partial x^1}(p) + \dots + a^n \frac{\partial f}{\partial x^n}(p)$. So in local coordinates, a vector at a point $p$ is fully described by how it differentiates functions along coordinate directions.

### Example

Let $f(x, y) = x^2 + x y$ and let $p = (1, 2)$. Define a vector field $X_q = 3 y\frac{\partial}{\partial x} - x\frac{\partial}{\partial y}$, for point $q = (x, y)$.  
Therefore the derivation (vector) at $p$, $X_p$ is $3 \cdot 2 \frac{\partial}{\partial x} - \frac{\partial}{\partial y} = 6 \frac{\partial}{\partial x} - \frac{\partial}{\partial y}$.  
We compute the partial derivatives, $\frac{\partial f}{\partial x} = 2x + y$, and $\frac{\partial f}{\partial y} = x$. At the point $p = (1, 2)$, we get $\frac{\partial f}{\partial x}(p) = 4$ and $\frac{\partial f}{\partial y}(p) = 1$.  
Thus, $X_p(f) = 6 \cdot 4 - 1 \cdot 1 = 23$.  

Picking another point $q = (-2, 0)$, we obtain another value for the application of the vector field $X$ to $f$, $X_q(f) = (3 \cdot 0 \frac{\partial}{\partial x} + 2 \frac{\partial}{\partial y})(f)(q) = -4$.

### Summary

Vectors on manifolds are best understood as **derivations** — operators acting on smooth functions, satisfying the Leibniz rule. The coordinate derivations $\frac{\partial}{\partial x^i}$ provide a natural basis for $T_pM$, and every vector can be written as a combination of them. This formalism is local, intrinsic, and independent of any embedding into Euclidean space.

This new view becomes extremely powerful when combined with differential forms, because it provides the foundation for defining integration, exterior differentiation, and the general machinery of geometric analysis on manifolds.

## Differential forms

Differential forms are the next level up from vector fields, a differential form is defined as a linear functional on the space of vector fields. In other words, a differential 1-form takes a vector field $X$ and produces a real number $\mathbb{R}$ for each point on the manifold $M$. Likewise, a differential $k$-form takes $k$ vector fields and produces a real number for every point on the manifold. Also, a differential 0-form is defined to be just a scalar function, that is a function from the manifold $M$ to real numbers. Intuitively, a differential form measures how well the vector field aligns with the form at some point on the manifold. A good way of visualizing differential forms in low dimensions can be found for example [here by Dan Piponi](http://yaroslavvb.com/papers/notes/piponi-on.pdf)(1998). 

### Differential form basis

A differential 1-form can look for example something like this, $\omega = 2 dx - 5xdy$, the quantities $dx$ and $dy$ are the dual 1-forms of the coordinate directions $\frac{\partial}{\partial x}, \frac{\partial}{\partial y}$, they are defined to be such that the application of the basis form to the corresponding coordinate basis vector produces simply $1$ for all points $p \in M$. This hints to the true fact that like vector fields, 1-forms can be made formed by a simple linear combination of those basis 1-forms.
 
Since there can be however differential forms with higher degrees, for example $2$-forms, we can use the wedge product ($\wedge$) to combine lower degree forms to build higher ones. For example we can have a 2-form on 3-dimensional manifold, like $\eta = 2 dx \wedge dy - 7 xdy \wedge dz + xy dx \wedge dz$. The details of the wedge product are left outside of this blog post for brevity, but details can be found on any standard differential geometry book.

### Differential form example

For a motivating example, the application of a 1-form $\omega = 2\,dx - 5x\,dy$.
Taking the point $p = (1, 3)$ and a vector field, $X = y \frac{\partial}{\partial x} + x \frac{\partial}{\partial y}$. At point $p$, the specific vector is $X_p = 3 \frac{\partial}{\partial x} + 1 \frac{\partial}{\partial y}$.  
Then we apply the 1-form $\omega$ to this vector, obtaining $\omega(X_p) = 2 \cdot 3 - 5 \cdot 1 \cdot 1 = 6 - 5 = 1$.  
So at the point $p = (1, 3)$, the 1-form $\omega = 2 dx - 5x dy$ evaluates to $1$ when applied to the vector field $X$.

If we drop the specific point $p$ but rather look at the 1-form 

This gives an idea of how 1-forms act like "detectors" or "probes" that measure the components of vector fields, weighted by their coefficients and position on the manifold.

### Differential 2-form example

Let us now consider a differential 2-form, which takes in two vectors and produces a real number. A typical example in $\mathbb{R}^3$ might look like:$\eta = x\,dy \wedge dz + y\,dz \wedge dx + z\,dx \wedge dy$

Evaluated on a pair of vectors $v = \frac{\partial}{\partial y} + \frac{\partial}{\partial z}, w = \frac{\partial}{\partial x} + 2\frac{\partial}{\partial z}$, the value of the 2-form at point $p = (1, 2, 3)$ is computed using the wedge products.

$dy \wedge dz (v, w)$:
$$
dy \wedge dz (v, w) = \det \begin{bmatrix} v^y & v^z \\ w^y & w^z \end{bmatrix} = \det \begin{bmatrix} 1 & 1 \\ 0 & 2 \end{bmatrix} = 2
$$

$dz \wedge dx (v, w)$:
$$
\det \begin{bmatrix} v^z & v^x \\ w^z & w^x \end{bmatrix} = \det \begin{bmatrix} 1 & 0 \\ 2 & 1 \end{bmatrix} = 1
$$

$dx \wedge dy (v, w)$:
$$
\det \begin{bmatrix} v^x & v^y \\ w^x & w^y \end{bmatrix} = \det \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} = -1
$$

Now we evaluate $\eta(v, w)$ at $p = (1, 2, 3)$:
- The first term gives $x \cdot dy \wedge dz (v, w) = 1 \cdot 2 = 2$
- The second term gives $y \cdot dz \wedge dx (v, w) = 2 \cdot 1 = 2$
- The third term gives $z \cdot dx \wedge dy (v, w) = 3 \cdot (-1) = -3$

Putting it together:

$$
\eta(v, w) = 2 + 2 - 3 = 1
$$

So the 2-form $\eta$ evaluates to 1 on the vectors $v$ and $w$ at the point $(1, 2, 3)$.




## Dirac-delta forms and restating Plateau's problem

Having introduced differential forms and their role as intrinsic, coordinate-free tools for probing vector fields and oriented geometry, we are now ready to discuss a class of generalized forms that plays a central role in reformulating the Plateau problem: **Dirac-delta differential forms**. These are not smooth forms in the usual sense, but rather distributions—or more precisely, **currents**, which are continuous linear functionals on the space of smooth compactly supported differential forms. They allow us to represent highly singular geometric objects, such as curves or surfaces, as objects that can still be integrated against smooth test forms.

The key idea is to represent a surface $S \subset M$ not by an explicit parametrization or an embedding, but by the **1-form current** $\delta_S$, which satisfies the identity
$$
\int_M \omega \wedge \delta_S = \int_S \omega
$$
for any smooth 2-form $\omega$ on the ambient manifold $M$. This object behaves like a "generalized differential form" that is zero almost everywhere, but supported entirely on the surface $S$. It captures the geometry of $S$ in the weak sense—via its action on test forms—making it ideal for variational formulations where smoothness may not be guaranteed.

Intuitively, we can visualize dirac-delta forms $\delta$ representing curves and surfaces as "impulses". As normal differential forms can be visualized as vector fields using [musical isomorphisms]() in 3-dimensions, these dirac-delta forms can be visualized in a similar way as being connected to vector fields but somehow being more "local". This locality can be realized as the vector field vanishing as we stray too far from the associated curve or surface.


# Visualization of dirac-delta forms

Here we have a couple of different visualization of dirac-delta differential forms, as stated previously, the dirac-delta forms are differential forms in the weak sense that they are localized to a specific submanifold. In our setting of 3-dimensional euclidean space, the ambient space $M = \mathbb{R}^3$ and therefore a curve $\Gamma$ can be realized using a dirac delta $2$-form. Similarly, a 2-dimensional surface $S$ can be realized as a dirac-delta $1$-form.

## Representation of a curve

In [3]:
%%manim -ql -v WARNING -o curve_dirac_delta.mp4 scene
# Instantiate the grid
N = 64
grid = Grid(resolution=N)

# Instantiate some example curve
curve_function = lambda t: ellipse_function(t, a = 0.4, b = 0.4, base_point=[0.5, 0.5, 0.5]) + np.array([0, 0, 0.2 * np.sin(t * 4 * np.pi + 0.5 * np.pi)])

# Actual points of the curve
t_range = np.linspace(0, 1, grid.res[0])
curve = np.array([curve_function(t) for t in t_range])
epsilon = 0.1

# Visualize the curve
field = VectorField(grid=grid)
def dirac_delta_form(point: np.array):
   # Finding the closest point to the curve
   q, t_closest = closest_point_on_curve(point, curve)
   
   # Tangent vector value in the point on the curve
   if np.sum((q - point) ** 2) <= (epsilon ** 2):
      tangent = numerical_tangent_function(t_range[t_closest], curve_function, h = 0.001)
      tangent_norm = tangent / (np.linalg.norm(tangent))
      return tangent_norm * (1.0 / (np.pi * epsilon ** 2)) # Coefficient 0.1 is for visualization purposes

   return np.zeros_like(point)
field.set_vectors_from_function(dirac_delta_form)
print("building the scene...")
scene = build_vector_field_scene(grid, field, curve_function=curve_function)


building the scene...


Manim Community v0.18.1

In the above plot, we have a visualization of a dirac-delta $2$-form, representing a curve $\Gamma$. The curve is displayed as a black solid line, while the dirac-delta curve representation is displayed as blue streamlines.

The visualization of the dirac-delta form is based on the musical isomorphism correspondence with the curve tangent vector field. More details on how this visualization is made can be found in the original paper.  
This mental image is very helpful to keep in mind in the next sections as we dive into the actual implementation, the key idea being that we want to __represent a geometric object as a localized differential form__.

In [4]:
%%manim -ql -v WARNING -o surface_dirac_delta.mp4 surface_scene
# Instantiate the grid
N = 16
grid = Grid(resolution=N)
epsilon = 0.1
# Surface representation
input_points = grid.get_flat_positions()[:, :2]

surface_function = lambda x, y: np.array([x, y, 0.5 + 0.1 * np.sin(x * 2 * np.pi) + 0.1 * np.cos(y * x * 2 * np.pi)])
surface = np.array([surface_function(pos[0], pos[1]) for pos in input_points])

def compute_surface_normal(position: np.array) -> np.array:
    x, y, z = position
    x_partial = numerical_tangent_function(x, lambda t: surface_function(t, y), h = 0.001)
    y_partial = numerical_tangent_function(y, lambda t: surface_function(x, t), h = 0.001)
    return np.cross(x_partial, y_partial)

def surface_form(point: np.array) -> np.array:
    q, t_star = closest_point_on_surface(p = point, surface_points=surface)

    # Tangent vector value in the point on the curve
    if np.sum((q - point) ** 2) <= (epsilon ** 2):
        normal = compute_surface_normal(q)
        return normal * (1.0 / (np.pi * epsilon ** 2)) # Coefficient 0.1 is for visualization purposes
    return np.zeros_like(point)

field = VectorField(grid = grid)
field.set_vectors_from_function(surface_form)
surface_scene = build_vector_field_scene(grid=Grid, vector_field=field, surface_function=surface_function)


Manim Community v0.18.1

Here we have a similar plot for a two dimensional surface embedded in 3-dimensional space, again the surface itself is displayed in black and the dirac-delta form representation is visualized with blue streamlines.

You can see that close to the surface, the streamlines follow normal vectors of the surface, this means that the differential form somehow encodes the information of the surface using these normals.  
More details also on this can be found in the original paper.

# Recap

In this blog post we went through a lot of the underlying mathematical tools we use in order to solve the Plateau problem.  
We discovered how from vector fields we can arrive at more intrinsic differential geometric objects and how to represent different geometric objects using more abstract objects.

In the next part we will get our hands dirty and start thinking about the actual optimization problem and then move on to actual implementation in the following posts.